In [1]:
import gradio as gr
from pathlib import Path
import numpy as np
import cv2 as cv2
import pickle
import face_recognition
from statistics import mode
from tqdm import tqdm
from py_files import add_new_person as anp
import glob
from threading import Lock, Thread

import warnings
warnings.filterwarnings("ignore")

/Users/ericcriteser/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-21 09:01:01.287906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open('data/processed/lfw_names.pickle', 'rb') as handle:
    names = pickle.load(handle)
    
with open('data/processed/lfw_encodings.pickle', 'rb') as handle:
    face_encodings = pickle.load(handle)

In [3]:
def save_new_data(t_e, t_n):
    for t in t_n:
        names.append(t)
    for e in t_e:
        face_encodings.append(e)
    assert len(names) == len(face_encodings)
    anp.save_data(names, 'data/processed/lfw_names.pickle')
    anp.save_data(face_encodings, 'data/processed/lfw_encodings.pickle')

In [4]:
import time
def new_person(name, img):
    msg = 'Success'
    if name != '':
        try:
            resized_img = anp.resize_img(img)
            anp.create_directory(name)
            anp.mutate_img(resized_img, name)
            file_list = glob.glob('data/processed/temporary_mutated_images/{}/*'.format(name))
            t_encodings, t_names = anp.encodings_names(file_list)
            _ = save_new_data(t_encodings, t_names)
            anp.delete_directory(Path('data/processed/temporary_mutated_images/{}'.format(name))) 
        except:
            msg = 'Failed to process new image'
        return msg

demo = gr.Interface(fn = new_person, inputs = ['text', 'image'], outputs = 'text')

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


LEN FILE LIST 6
IMAGE SHAPE (400, 300, 3)
BOXES 1
IMAGE SHAPE (400, 300, 3)
BOXES 1
IMAGE SHAPE (400, 300, 3)
BOXES 1
IMAGE SHAPE (400, 300, 3)
BOXES 1
IMAGE SHAPE (400, 300, 3)
BOXES 1
IMAGE SHAPE (400, 300, 3)
BOXES 1
6
